<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2023/ex07notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex07notebookC

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2023)


----
## 事前学習済みニューラルネットを動かしてみよう
----




----
### 準備




#### GPUを利用するようにランタイムのタイプを変更する



この notebook の後半の画像生成のコードは，GPU を利用できる環境でないと動きません．次のようにしてランタイムのタイプを変更してから実行してください．

1. メニューの「ランタイム」 > 「ランタイムのタイプを変更」 を選択．
1. 「ノートブックの設定」というポップアップウィンドウが開くので，「ハードウェアアクセラレータ」を「None」から「GPU」に変更し，「保存」する
1. いつもどおりコードセルを実行する．すでに実行していた場合，「以前のランタイムを削除する」というポップアップウィンドウが現れるので，「OK」を押しし，一番最初のコードセルから実行し直ます．


#### いろいろ import

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

import io
import requests
from PIL import Image

In [ ]:
# PyTorch 関係のほげ
import torch
from torchvision import models
from torchvision.io.image import read_image
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

GPU が使えるようになっていれば，↑のセルを実行すると `cuda:0` と出力されるはずです．


----
### 実験1 物体検出

notebookB では，「一枚の画像全体に何が写っているかを認識する」識別問題を扱うニューラルネットを動かしましたが，ここでは，「一枚の画像の中に写っている複数の物体を検出する」問題を扱うニューラルネットを動かしてみましょう．このニューラルネットは，一枚の画像を入力すると，画像中の物体ごとに，その位置とそれが何であるかを表す情報を出力します．

ここでは，FasterRCNN と呼ばれる，畳み込みニューラルネットによる物体検出の仕組みの事前学習済みモデルを使います．
このモデルは，COCO (https://cocodataset.org/) と呼ばれる大規模画像データセットで 81 種類の物体の検出を学習したものです．


- Shaoqing Ren, Kaiming He, Ross Girshick, and Jian Sun, "Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks," NeurIPS 2015, https://arxiv.org/abs/1506.01497
- https://pytorch.org/vision/0.15/models/generated/torchvision.models.detection.fasterrcnn_resnet50_fpn.html


In [ ]:
# FasterRCNN の事前学習済みモデルを入手

weights = models.detection.FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
preprocess = weights.transforms()
model = models.detection.fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.9)
model = model.to(device)
model.eval()
#print(model)

# 物体のクラスの一覧を出力
n = 0
for cn in weights.meta["categories"]:
    if cn != 'N/A':
        print(n, cn, end='\n')
        n += 1

実験用のサンプル画像を入手します．これらは，COCOデータセットの学習用または評価用のデータの一部です．

In [ ]:
# 画像を入手

urlDict = {
    'dog':'https://farm6.staticflickr.com/5124/5379029845_60f6314172_z.jpg',
    'cats':'https://farm1.staticflickr.com/16/23200321_dcff6ba227_z.jpg',
    'carelephant':'https://farm8.staticflickr.com/7157/6822207699_71e174fd3f_z.jpg',
    'bicycles':'https://farm1.staticflickr.com/33/67728109_0d11a646ef_z.jpg',
    'teddybears':'https://farm4.staticflickr.com/3584/3554604954_2e01e4d007_z.jpg',
}

for key in urlDict.keys():
    rv = requests.get(urlDict[key])
    assert rv.status_code == 200, '画像のダウンロードに失敗しました'
    with io.BytesIO(rv.content) as buf:
        img = Image.open(buf)
        fn = f'{key}.png'
        print(fn)
        img.save(fn)

In [ ]:
#@title 物体検出
#@markdown 以下で 0 から 4 までの数をひとつ選んでからこのセルを実行すると，物体検出の結果が表示されます．
i = 0 #@param [0, 1, 2, 3, 4] {type: 'raw'}

key = list(urlDict.keys())[i]
fn = f'{key}.png'
print(fn)

# 画像を読み込んでモデルへの入力に加工
img = read_image(fn)
X = torch.unsqueeze(preprocess(img), axis=0).to(device)

# モデルの出力を計算
Y = model(X)[0]

# 出力の情報を画像に加工
labels = [weights.meta['categories'][i] for i in Y['labels']]
bbox = draw_bounding_boxes(img, boxes=Y['boxes'], labels=labels, colors='#00ff00', width=3)
imgResult = to_pil_image(bbox.detach())
imgResult.show()


次のセルを実行すると，適当な画像ファイルを Colab 上へアップロードできます．いろいろ試してみましょう．

In [ ]:
# Colab へファイルをアップロード
from google.colab import files
rv = files.upload()

# ファイル一覧
! ls

以下のセルの1行目のファイル名をアップロードしたものに変更してから実行すると，物体検出してくれます．

In [ ]:
fn = 'teddybears.png' # ここを拡張子含めてアップロードしたファイルの名前に変更

print(fn)
img = read_image(fn)
X = torch.unsqueeze(preprocess(img), axis=0).to(device)
Y = model(X)[0]
labels = [weights.meta['categories'][i] for i in Y['labels']]
bbox = draw_bounding_boxes(img, boxes=Y['boxes'], labels=labels, colors='#00ff00', width=3)
imgResult = to_pil_image(bbox.detach())
imgResult.show()

---
### 実験2 画像生成

次は，Stable Diffusion と呼ばれる画像生成モデルを動かしてみます． Stable Diffusion は，
[Stability AI](https://ja.stability.ai/)が開発・提供しているものです．
ウェブ上で試用できます（ https://stablediffusionweb.com/ )が，学習済みモデルが公開されていますので，それを入手して自分のPCで動かしてみることもできます．ここでは， Colab 上で動かしてみましょう．

- Stable Diffusion の中心にある Latent Diffusion Model に関する論文: 
R. Rombach, A. Blattmann, D. Lorenz, P. Esser, B Ommer, ''[High-Resolution Image Synthesis with Latent Diffusion Models](https://arxiv.org/abs/2112.10752)'', CVPR2022
- Wikipedia の記事: https://ja.wikipedia.org/wiki/Stable_Diffusion

準備として，標準の Colab の環境にはインストールされていないソフトウェアパッケージをいくつか追加でインストールします．

In [ ]:
! pip install --upgrade git+https://github.com/huggingface/diffusers.git transformers accelerate scipy torch

事前学習済みモデルをダウンロードします．巨大なモデルでパラメータ数が膨大なので，少し時間がかかります．

In [ ]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch

model_id = 'stabilityai/stable-diffusion-2'

# ノイズスケジューラの設定
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder='scheduler')
# モデルとそのパラメータの入手
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, revision='fp16', torch_dtype=torch.float16)
# GPU を使う設定にする
assert torch.cuda.is_available(), 'この実験はGPUが使える環境でないとできません'
pipe = pipe.to('cuda')

以下の `prompt` に記したテキスト（プロンプト）をもとに画像を生成します．ランダム性があるので，同じプロンプトでも実行の度に生成結果は変わります．何度か実行してみるとよいでしょう．

In [ ]:
prompt = 'a photo of an astronaut riding a horse on mars'
image = pipe(prompt).images[0]
image

引数でいろいろ指定できます．詳しいことが知りたいひとは，リンク先のドキュメントを参照してください．

https://huggingface.co/docs/diffusers/v0.16.0/en/api/pipelines/stable_diffusion/text2img



In [ ]:
prompt = 'a photo of a man facing his laptop computer and a cat sleeping on the laptop computer'
neg_prompt = 'black cat'
image = pipe(prompt, negative_prompt=neg_prompt, guidance_scale=7.5).images[0]
image

In [ ]:
prompt = 'a figure of mount Fuji erupting with flash of lightning, plume and lava'
neg_prompt = None
image = pipe(prompt, negative_prompt=neg_prompt, guidance_scale=15.0).images[0]
image

自分で適当なプロンプトを指定していろいろ試してみてね．